#### Pydantic Basics: Creating and Using Models
Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.



### What are data class?

- dataclasses are a feature in Python (introduced in Python 3.7) that allow you to create classes for storing data with much less boilerplate code. They automatically generate special methods like __init__, __repr__, __eq__, and others, based on the class attributes you define.


### Difference between data class and normal class?
- Dataclasses are ideal for simple, structured data containers—less code, less error, built-in best practices.
- Normal classes are more flexible for complex logic and behavior, but require more code for “data holder” functionality.
- Use when your class is primarily for storing data (think: records, config objects, simple model parameters). Fast to write, easy to maintain, less error-prone.
- Use when your class has a lot of custom logic, methods, state changes, or needs to control its own construction beyond just holding data.


In [1]:
from dataclasses import dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str

In [2]:
person=Person(name="Krish",age=35,city="Bangalore")
print(person)

Person(name='Krish', age=35, city='Bangalore')


In [3]:
person=Person(name="Krish",age=35,city=35)
print(person)

Person(name='Krish', age=35, city=35)


here, there is no type check is happening in case of city in above example

to solve this Pydantic is used

In [5]:
## Pydantic
from pydantic import BaseModel

In [ ]:
# this is called as the data module, since we are using BaseModel

class Person1(BaseModel):
    name:str
    age:int
    city:str

person=Person1(name="Krish",age=35,city="Bangalore")
print(person)

name='Krish' age=35 city='Bangalore'


In [9]:
# person1=Person1(name="Krish",age=35,city=35)
# print(person1)
# this will give ValidationError

In [10]:
person2=Person1(name="Krish",age=35,city="35")
print(person2)

name='Krish' age=35 city='35'


#### 2. Model with Optional Fields
Add optional fields using Python's Optional type:



In [11]:
from typing import Optional
class Employee(BaseModel):
    id:int
    name:str
    department:str
    salary: Optional[float]=None        # Optional with default value
    is_active: Optional[bool]=True      # Optional field with default value

    

In [13]:
emp1=Employee(id=1,name="John",department="CS")
print(emp1)

# type casting happens automatically, if we try salary="30000", it will convert it in float

id=1 name='John' department='CS' salary=None is_active=True


In [14]:
emp2=Employee(id=2,name="Krish",department="CS",salary="30000")
print(emp2)

id=2 name='Krish' department='CS' salary=30000.0 is_active=True


Definition:
- Optional[type]: Indicates the field can be None

- Default value (= None or = True): Makes the field optional

- Required fields must still be provided

- Pydantic validates types even for optional fields when values are provided

In [ ]:
emp3=Employee(id=2,name="Krish",department="CS",salary="30000",is_active=1)
print(emp3)

# is_active=23, will give ValidationError
# boolean value takes 0 [False] and 1 [True]

id=2 name='Krish' department='CS' salary=30000.0 is_active=True


In [16]:
from typing import List

# whenever we inherit BaseModel, that means data validation is applied and Pydantic will start it's work

class Classroom(BaseModel):
    room_number:str
    students: List[str] #List of strings
    capacity:int

In [ ]:
# Create a classroom
classroom = Classroom(
    room_number="A101",
    students=("Alice", "Bob", "Charlie"),
    capacity=30
)
print(classroom)

# this works, as tuple is type casting to list
# pydantic is written in Rust, so it's fast

room_number='A101' students=['Alice', 'Bob', 'Charlie'] capacity=30


In [ ]:
list(("Alice", "Bob", "Charlie"))

# type casting tuple to list

['Alice', 'Bob', 'Charlie']

In [20]:
# Create a classroom
# classroom1 = Classroom(
#     room_number="A101",
#     students=("Alice", 123, "Charlie"),
#     capacity=30
# )
# print(classroom1)

In [21]:
try:
    invalid_val=Classroom(room_number="A1",students=["Krish",123],capacity=30)

except Exception as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


#### 4. Model with Nested Models
Create complex structures with nested models:

In [25]:
from pydantic import BaseModel

class Address(BaseModel):
    street:str
    city:str
    zip_code:str

class Customer(BaseModel):
    customer_id:int
    name:str
    address:Address  ## Nested Model

In [26]:
customer=Customer(customer_id=1,name="Krish",
                  address={"street":"Main street","city":"Boston","zip_code":"02108"})

print(customer)

customer_id=1 name='Krish' address=Address(street='Main street', city='Boston', zip_code='02108')


In [22]:
# customer=Customer(customer_id=1,name="Krish",
#                   address={"street":"Main street","city":123,"zip_code":"02108"})

# print(customer)

#### Pydantic Fields: Customization and Constraints

The Field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values, aliases, and more. Here's a comprehensive tutorial with examples.

In [23]:
from pydantic import BaseModel,Field

class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float=Field(gt=0,le=10000)  ## greater than 0 and less than or equal to 1000
    quantity:int=Field(ge=0)

# item=Item(name="Book", price=100000,quantity=10)
# print(item)

item=Item(name="Book", price=100,quantity=10)
print(item)

name='Book' price=100.0 quantity=10


In [24]:
class User(BaseModel):
    username:str=Field(description="Unique username for the user")
    age:int=Field(default=18,description="User age default to 18 ")
    email:str= Field(default_factory=lambda: "user@example.com",description="Default email address")


# Examples
user1 = User(username="alice")
print(user1)

# we are giving default value with default_factory

username='alice' age=18 email='user@example.com'


In [ ]:
user2 = User(username="bob", age=25, email="bob@domain.in")
print(user2)

username='bob' age=25 email='bob@domain..in'


In [ ]:
User.model_json_schema()

# entire model schema is available now just like swagger in FastAPI

{'properties': {'username': {'description': 'Unique username for the user',
   'title': 'Username',
   'type': 'string'},
  'age': {'default': 18,
   'description': 'User age default to 18 ',
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'Default email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['username'],
 'title': 'User',
 'type': 'object'}

In [ ]:
# type casting vs type hinting